In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2024,1)

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')
    ivt=ds['ivt']
    
    ds=xr.open_dataset(dir3+'jra55_iwv_'+str(year)+'.nc')
    iwv=ds['iwv']
    
    ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    labeled_interp=labeled/labeled
    labeled_interp=labeled_interp.fillna(0).interp_like(s)
    labeled_interp=labeled_interp.where(labeled_interp>=0.5)
    labeled_interp=labeled_interp/labeled_interp
    
    ivt['lat']=labeled.lat.values
    ivt['lon']=labeled.lon.values
    
    iwv['lat']=labeled.lat.values
    iwv['lon']=labeled.lon.values
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.mean(dim='time',skipna=True)
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.mean(dim='time',skipna=True)
    s_sel=s.where(labeled_interp>0)
    s_mean=s_sel.mean(dim='time',skipna=True)
    #mfc_sel=mfc.where(labeled>0)
    #mfc_mean=mfc_sel.mean(dim='time',skipna=True)
    
    if i==0:
        ivt_concat_jra55_polar=ivt_mean
        iwv_concat_jra55_polar=iwv_mean
        s_concat_jra55_polar=s_mean
    else:
        ivt_concat_jra55_polar=xr.concat([ivt_concat_jra55_polar,ivt_mean],dim='time')
        iwv_concat_jra55_polar=xr.concat([iwv_concat_jra55_polar,iwv_mean],dim='time')
        s_concat_jra55_polar=xr.concat([s_concat_jra55_polar,s_mean],dim='time')
    
    
    ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    labeled_interp=labeled/labeled
    labeled_interp=labeled_interp.fillna(0).interp_like(s)
    labeled_interp=labeled_interp.where(labeled_interp>=0.5)
    labeled_interp=labeled_interp/labeled_interp
    
    ivt['lat']=labeled.lat.values
    ivt['lon']=labeled.lon.values
    
    iwv['lat']=labeled.lat.values
    iwv['lon']=labeled.lon.values
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.mean(dim='time',skipna=True)
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.mean(dim='time',skipna=True)
    s_sel=s.where(labeled_interp>0)
    s_mean=s_sel.mean(dim='time',skipna=True)
    #mfc_sel=mfc.where(labeled>0)
    #mfc_mean=mfc_sel.mean(dim='time',skipna=True)
    
    if i==0:
        ivt_concat_jra55_fixed=ivt_mean
        iwv_concat_jra55_fixed=iwv_mean
        s_concat_jra55_fixed=s_mean
    else:
        ivt_concat_jra55_fixed=xr.concat([ivt_concat_jra55_fixed,ivt_mean],dim='time')
        iwv_concat_jra55_fixed=xr.concat([iwv_concat_jra55_fixed,iwv_mean],dim='time')
        s_concat_jra55_fixed=xr.concat([s_concat_jra55_fixed,s_mean],dim='time')

    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    labeled_interp=labeled/labeled
    labeled_interp=labeled_interp.fillna(0).interp_like(s)
    labeled_interp=labeled_interp.where(labeled_interp>=0.5)
    labeled_interp=labeled_interp/labeled_interp
    
    ivt['lat']=labeled.lat.values
    ivt['lon']=labeled.lon.values
    
    iwv['lat']=labeled.lat.values
    iwv['lon']=labeled.lon.values
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.mean(dim='time',skipna=True)
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.mean(dim='time',skipna=True)
    s_sel=s.where(labeled_interp>0)
    s_mean=s_sel.mean(dim='time',skipna=True)
    #mfc_sel=mfc.where(labeled>0)
    #mfc_mean=mfc_sel.mean(dim='time',skipna=True)
    
    if i==0:
        ivt_concat_jra55_fixed_poleward=ivt_mean
        iwv_concat_jra55_fixed_poleward=iwv_mean
        s_concat_jra55_fixed_poleward=s_mean
    else:
        ivt_concat_jra55_fixed_poleward=xr.concat([ivt_concat_jra55_fixed_poleward,ivt_mean],dim='time')
        iwv_concat_jra55_fixed_poleward=xr.concat([iwv_concat_jra55_fixed_poleward,iwv_mean],dim='time')
        s_concat_jra55_fixed_poleward=xr.concat([s_concat_jra55_fixed_poleward,s_mean],dim='time')

In [ ]:
dk=xr.Dataset()
dk['ivt_concat_jra55_variable']=(('time','lat','lon'),ivt_concat_jra55_polar.values)
dk['iwv_concat_jra55_variable']=(('time','lat','lon'),iwv_concat_jra55_polar.values)
dk['s_concat_jra55_variable']=(('time','lat1','lon1'),s_concat_jra55_polar.values)
#dk['mfc_concat_merra2_variable']=(('time','lat','lon'),mfc_concat_merra2_polar.values)

dk['ivt_concat_jra55_fixed']=(('time','lat','lon'),ivt_concat_jra55_fixed.values)
dk['iwv_concat_jra55_fixed']=(('time','lat','lon'),iwv_concat_jra55_fixed.values)
dk['s_concat_jra55_fixed']=(('time','lat1','lon1'),s_concat_jra55_fixed.values)
#dk['mfc_concat_merra2_fixed']=(('time','lat','lon'),mfc_concat_merra2_fixed.values)

dk['ivt_concat_jra55_fixed_poleward']=(('time','lat','lon'),ivt_concat_jra55_fixed_poleward.values)
dk['iwv_concat_jra55_fixed_poleward']=(('time','lat','lon'),iwv_concat_jra55_fixed_poleward.values)
dk['s_concat_jra55_fixed_poleward']=(('time','lat1','lon1'),s_concat_jra55_fixed_poleward.values)
#dk['mfc_concat_merra2_fixed_poleward']=(('time','lat','lon'),mfc_concat_merra2_fixed_poleward.values)

dk.coords['time']=np.arange(1980,2024,1)
dk.coords['lat']=ivt_concat_jra55_polar.lat.values
dk.coords['lon']=ivt_concat_jra55_polar.lon.values
dk.coords['lat1']=s_concat_jra55_polar.lat.values
dk.coords['lon1']=s_concat_jra55_polar.lon.values
dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'
dk['lat1'].attrs["units"]='degrees_north'
dk['lon1'].attrs["units"]='degrees_east'
dk['time'].attrs["units"]='years'

try:
    os.remove(dir_data+'ar_vars_geomeans_jra55.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_vars_geomeans_jra55.nc',mode='w',format='NETCDF4')